In [ ]:
!apt-get install -y git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
! rm -rf NLP-project
! git clone https://github.com/ReviBa/NLP-project
! cp -rf NLP-project/* .

In [ ]:
! pip install --q -r requirments.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 61.0 MB/s eta 0:

In [ ]:
from t5.train import t5_train
from utils.tokens_utils import get_tokenizer_based_on_data, get_max_tokens_number
from context_manager import DialogContextManager
from t5.model import T5Model
import pandas as pd
import warnings
import torch

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
assert DEVICE == 'cuda'

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Using regular T5 was causing problems. he was trained on different languages,
# so sometimes it returned results that are not in english.
MODEL_NAME = "google/t5-v1_1-base"
BATCH_SIZE = 4
PATH = "/content/NLP-project/resources/embeddings_comparison_mini_cosine.csv"
SAVE_TO_CHECKPOINT = "style_transfer_model-10-epochs-on-new-data"

In [ ]:
style_transfer_df = pd.read_csv(PATH)
style_transfer_df = style_transfer_df[style_transfer_df['cosine']<=0.45]
cols = ['input', 'target']
train_df =  style_transfer_df.rename(columns={'sentence': cols[0], 'all-MiniLM-L6-v2 with cosine': cols[1]})[cols].drop_duplicates()


In [ ]:
train_df.count()

input     27152
target    27152
dtype: int64

In [ ]:
tokenizer = get_tokenizer_based_on_data(train_df, *cols)
i_max_token_len, o_max_token_len = get_max_tokens_number(train_df, *cols)

In [ ]:
! export CUDA_LAUNCH_BLOCKING=1
! export TORCH_USE_CUDA_DSA=1
trained_model, trainer = t5_train(train_df, DEVICE, tokenizer,
                                  MODEL_NAME, i_max_token_len, o_max_token_len, train_batch_size=BATCH_SIZE,
                                  test_batch_size=BATCH_SIZE, epochs=2, save_to_checkpoint=SAVE_TO_CHECKPOINT, train_from_checkpoint="/content/drive/MyDrive/style_transfer_model-8-epochs-on-new-data.ckpt")

KeyboardInterrupt: ignored

In [ ]:

style_transfer_df = pd.read_csv(PATH)
cols = ['input', 'target']
train_df =  style_transfer_df.rename(columns={'sentence': cols[0], 'all-MiniLM-L6-v2 with cosine': cols[1]})[cols].drop_duplicates()

# List of strings to search for
search_list = ['Scranton', 'Michael Scott', 'Dunder-Mifflin', 'Dunder Mifflin', 'Jim', 'Pam', 'Dwight','Holly', 'Jan']

# Filtering the DataFrame
filtered_df = train_df[train_df['target'].str.contains('|'.join(search_list), na=False)]

In [ ]:
filtered_df

,input,target
67,"Certainly , Mr . Emory . Is there a problem ,...","Dwight, are you talking about us?"
186,"We put $ 500 into the savings account , and $...",Jan called this morning and pledged $500.00.
299,"Hello , Ann ! Is that you ?","Hi, Jan!"
365,You'll be happy to know that it's open 24/7 .,It is Friday morning and it is another beautif...
385,"Say , Jim , how about going for a few beers af...","Jim, good party, huh? Just a little something ..."
...,...,...
69446,John ? I have to say I was bothered by the way...,Am I worried that Jim and David are having a m...
69550,"Ten gallons of premium . That's $ 18.56 , ple...",I need you to get me the prices that you're c...
69575,"Jim and you are bosom friends , aren't you ?",Jim and I are great friends
69663,"Hi , Ben . Come in . Diane's not ready yet , ...","Pam, can you come in here, please? Have a seat"


In [ ]:
trained_model, trainer = t5_train(filtered_df, DEVICE, tokenizer,
                                  MODEL_NAME, i_max_token_len, o_max_token_len, train_batch_size=BATCH_SIZE,
                                  test_batch_size=BATCH_SIZE, epochs=3, save_to_checkpoint="style_transfer_model-9-epochs-on-new-data-and-more", train_from_checkpoint="/content/drive/MyDrive/style_transfer_model-10-epochs-on-new-data.ckpt")

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 254 M 
-----------------------------------------------------
254 M     Trainable params
0         Non-trainable params
254 M     Total params
1,019.717 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 0, global step 210: val_loss reached 0.35780 (best 0.35780), saving model to "/content/drive/MyDrive/style_transfer_model-9-epochs-on-new-data-and-more.ckpt" as top 2


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 1, global step 421: val_loss reached 0.30678 (best 0.30678), saving model to "/content/drive/MyDrive/style_transfer_model-9-epochs-on-new-data-and-more-v1.ckpt" as top 2


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 2, global step 632: val_loss reached 0.29412 (best 0.29412), saving model to "/content/drive/MyDrive/style_transfer_model-9-epochs-on-new-data-and-more.ckpt" as top 2


In [ ]:
dd_model = T5Model("t5-base")
dd_model.model.resize_token_embeddings(32128)
checkpoint = torch.load("/content/drive/MyDrive/NLP project/Option3-trained-on-daily-dialog-splitted-corpus-second-epoch.ckpt")
dd_model.load_state_dict(checkpoint['state_dict'])
dd_model.to(DEVICE)

# trained_model = T5Model(MODEL_NAME)
# trained_model.load_from_checkpoint_with_custom_tokenizer(tokenizer, "/content/drive/MyDrive/style_transfer_model-10-epochs-on-new-data-and-more.ckpt")
trained_model.to(DEVICE)

T5Model(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(36914, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(36914, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
                (wi_1): Linear(in_fe

In [ ]:
context_manager = DialogContextManager(dd_model, tokenizer, i_max_token_len, DEVICE ,trained_model)

In [ ]:
context_manager.write("Where do you work?", debug=True)

generic response:  Person2:I work in the forestry department.


'I am the regional manager of Dunder-Mifflin, Michael Scott'

In [ ]:
context_manager.context

['Person1:Who is your girlfriend?', 'Holly Flax']

In [ ]:
trainer.save_checkpoint("/content/style_transfer_model-9-epochs-on-new-data-and-more.ckpt")